## 文本情感分类
使用文本情感分类来分析文本作者的情绪。这个问题也叫情感分析，并有着广泛的应用。例如，我们可以分析用户对产品的评论并统计用户的满意度，或者分析用户对市场行情的情绪并用以预测接下来的行情。

### 1 使用循环神经网络
应用预训练的词向量和含多个隐藏层的双向循环神经网络，来判断一段不定长的文本序列中包含的是正面还是负面的情绪。

In [35]:
import collections
import os
import random
import tarfile
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data
import torch.nn.functional as F
import my_utils

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DATA_ROOT = '../data'

In [3]:
fname = os.path.join(DATA_ROOT, 'aclImdb_v1.tar.gz')
with tarfile.open(fname, 'r') as f:
    f.extractall(DATA_ROOT)

In [4]:
from tqdm import tqdm

def read_imdb(folder='train', data_root=os.path.join(DATA_ROOT, 'aclImdb')):
    data = []
    for label in ['pos', 'neg']:
        folder_name = os.path.join(data_root, folder, label)
        for file in tqdm(os.listdir(folder_name)):
            with open(os.path.join(folder_name, file), 'rb') as f:
                # 每个样本是一条评论以及对应的标签
                review = f.read().decode('utf-8').replace('\n', '').lower()
                data.append([review, 1 if label == 'pos' else 0])
    random.shuffle(data)
    return data

In [5]:
train_data, test_data = read_imdb('train'), read_imdb('test')

100%|██████████| 12500/12500 [00:00<00:00, 26258.24it/s]


**数据预处理**：

对每条评论基于空格进行分词。

In [7]:
def get_tokenized(data):
    """
    data是一个list，每个元素为[review, 0/1]，其中review是一个sequence
    """
    def tokenizer(text):
        return [tk.lower() for tk in text.split(' ')]
    return [tokenizer(review) for review, _ in data]

根据分好词的训练数据集创建词典（过滤掉了出现次数少于5的词）。

In [9]:
def get_vocab_imbd(data):
    tokenized_data = get_tokenized(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    # 调用Vocab.Vocab创建词典
    return Vocab.Vocab(counter, min_freq=5)

vocab = get_vocab_imbd(train_data)
len(vocab), type(vocab)

(46152, torchtext.vocab.Vocab)

因为每条评论长度不一致所以不能直接组合成小批量，我们定义preprocess_imdb函数对每条评论进行分词，并通过词典转换成词索引，然后通过截断或者补0来将每条评论长度固定成500。

In [10]:
def preprocess_imdb(data, vocab):
    max_l = 500
    def pad(x):
        # 输入的x是词的index组成的list
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))
    
    tokenized_data = get_tokenized(data)
    features = torch.tensor([
        pad([vocab.stoi[word] for word in words]) for words in tokenized_data
    ])
    labels = torch.tensor([score for _, score in data])
    return features, labels

In [11]:
batch_size = 64
train_set = Data.TensorDataset(*preprocess_imdb(train_data, vocab))
test_set = Data.TensorDataset(*preprocess_imdb(test_data, vocab))
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)

In [12]:
for X, y in train_iter:
    print('X', X.shape, 'y', y.shape)
    break
'#batches:', len(train_iter)

X torch.Size([64, 500]) y torch.Size([64])


('#batches:', 391)

**定义双向循环神经网络**：

每个词先通过嵌入层得到特征向量。然后，我们使用双向循环神经网络对特征序列进一步编码得到序列信息。最后，我们将编码的序列信息通过全连接层变换为输出。具体来说，我们可以将双向长短期记忆在最初时间步和最终时间步的隐藏状态连结，作为特征序列的表征传递给输出层分类。

In [13]:
class BiRNN(nn.Module):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers):
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size)
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embed_size, 
                               hidden_size=num_hiddens, 
                               num_layers=num_layers,
                               bidirectional=True)
        # 初始时间步和最终时间步的隐藏状态作为全连接层输入
        # 双向循环神经网络的隐藏层变量是num_hiddens*2，首尾都算上所以*4
        # 输出的是pos或者neg，因此是2
        self.decoder = nn.Linear(4*num_hiddens, 2)

    def forward(self, inputs):
        # inputs的形状是(批量大小, 词数=500)，因为LSTM需要将序列长度(seq_len)作为第一维，所以将输入转置后
        # 再提取词特征，输出形状为(词数, 批量大小, 词向量维度)
        # 这里的词数就是rnn中的num_steps，然后用embedding的方式取代了one-hot encoding
        embeddings = self.embedding(inputs.permute(1, 0))
        # rnn.LSTM只传入输入embeddings，因此只返回最后一层的隐藏层在各时间步的隐藏状态。
        # outputs形状是(词数, 批量大小, 2 * 隐藏单元个数)
        outputs, _ = self.encoder(embeddings) # output, (h, c)
        # 连结初始时间步和最终时间步的隐藏状态作为全连接层输入。它的形状为
        # (批量大小, 4 * 隐藏单元个数)。
        encoding = torch.cat((outputs[0], outputs[-1]), -1)
        outs = self.decoder(encoding)
        return outs

In [14]:
embed_size, num_hiddens, num_layers = 100, 100, 2
net = BiRNN(vocab, embed_size, num_hiddens, num_layers)

**加载预训练的词向量**：

In [15]:
glove_vocab = Vocab.GloVe(name='6B', dim=100, cache=os.path.join(DATA_ROOT, 'pretrained_glove'))

In [20]:
glove_vocab.vectors[0].shape[0]

100

用预训练好的词向量直接更新embedding的参数：

In [21]:
def load_pretrained_embedding(words, pretrained_vocab):
    """从预训练好的vocab中提取出words对应的词向量"""
    embed = torch.zeros(len(words), pretrained_vocab.vectors[0].shape[0]) # 初始化为0
    oov_count = 0 # out of vocabulary（预训练的词典没有这个词）
    for i, word in enumerate(words):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 1
    if oov_count > 0:
        print("There are %d oov words." % oov_count)
    return embed

In [22]:
net.embedding.weight.data.copy_(load_pretrained_embedding(vocab.itos, glove_vocab))
net.embedding.weight.requires_grad = False

There are 21202 oov words.


**训练模型**：

（因为太慢而没有执行本cell。。。）

In [ ]:
lr, num_epochs = 0.01, 1
# 要过滤掉不计算梯度的embedding参数
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
my_utils.general_train(net, train_iter, test_iter, loss, num_epochs, batch_size, 
                       params=None, lr=None, optimizer=optimizer)

**情感判断**：

In [24]:
def predict_sentiment(net, vocab, sentence):
    device = list(net.parameters())[0].device
    sentence = torch.tensor([vocab.stoi[word] for word in sentence], device=device)
    label = torch.argmax(net(sentence.view((1, -1))), dim=1)
    return 'positive' if label.item() == 1 else 'negative'

当前输出的判断是随机猜测的结果：

In [25]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great'])

'positive'

### 2 使用卷积神经网络（textCNN）

**一维互相关运算**：

![avatar](../resource/conv1d.svg)

In [26]:
def corr1d(X, K):
    w = K.shape[0]
    Y = torch.zeros((X.shape[0] - w + 1))
    for i in range(Y.shape[0]):
        Y[i] = (X[i: i + w] * K).sum()
    return Y

In [27]:
X, K = torch.tensor([0, 1, 2, 3, 4, 5, 6]), torch.tensor([1, 2])
corr1d(X, K)

tensor([ 2.,  5.,  8., 11., 14., 17.])

多输入通道的一维互相关运算（各个输入通道的结果相加）：

![avatar](../resource/conv1d-channel.svg)

In [28]:
def corr1d_multi_in(X, K):
    # 首先沿着X和K的第0维（通道维）遍历并计算一维互相关结果。然后将所有结果堆叠起来沿第0维累加
    return torch.stack([corr1d(x, k) for x, k in zip(X, K)]).sum(dim=0)

X = torch.tensor([[0, 1, 2, 3, 4, 5, 6],
              [1, 2, 3, 4, 5, 6, 7],
              [2, 3, 4, 5, 6, 7, 8]])
K = torch.tensor([[1, 2], [3, 4], [-1, -3]])
corr1d_multi_in(X, K)

tensor([ 2.,  8., 14., 20., 26., 32.])

多输入通道的一维互相关运算可以看作单输入通道的二维互相关运算：

![avatar](../resource/conv1d-2d.svg)

**时序最大池化层**：

textCNN中使用的时序最大池化（max-over-time pooling）层实际上对应一维全局最大池化层：假设输入包含多个通道，各通道由不同时间步上的数值组成，**各通道的输出即该通道所有时间步中最大的数值**。因此，时序最大池化层的输入在各个通道上的时间步数可以不同。

In [30]:
class GlobalMaxPool1d(nn.Module):
    def __init__(self):
        super(GlobalMaxPool1d, self).__init__()
        
    def forward(self, x):
        # x shape: (batch_size, channel, seq_len)
        # return shape: (batch_size, channel, 1)
        # seq_len就是时间步num_steps，也是特征的大小
        return F.max_pool1d(x, kernel_size=x.shape[2])

**textCNN模型**：

假设输入的文本序列由$n$个词组成，每个词用$d$维的词向量表示。那么输入样本的宽为$n$（一维向量的特征就是不同时间步上的元素），高为1，输入通道数为$d$（词向量的大小则是深度）。

三步走：

* 定义多个一维卷积核，并使用这些卷积核对输入分别做卷积计算。宽度不同的卷积核可能会捕捉到不同个数的相邻词的相关性。
* 对输出的所有通道分别做时序最大池化，再将这些通道的池化输出值连结为向量。
* 通过全连接层将连结后的向量变换为有关各类别的输出。这一步可以使用丢弃层应对过拟合。

下图解释了textCNN的设计。这里的输入是一个有11个词的句子，每个词用6维词向量表示。因此输入序列的宽为11，输入通道数为6。给定2个一维卷积核，核宽分别为2和4，输出通道数分别设为4和5。因此，一维卷积计算后，4个输出通道的宽为$11−2+1=10$，而其他5个通道的宽为$11−4+1=8$。尽管每个通道的宽不同，我们依然可以对各个通道做时序最大池化，并将9个通道的池化输出连结成一个9维向量。最终，使用全连接将9维向量变换为2维输出，即正面情感和负面情感的预测。

![avatar](../resource/textcnn.svg)

In [31]:
class TextCNN(nn.Module):
    def __init__(self, vocab, embed_size, kernel_sizes, num_channels):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size)
        # 不参与训练的嵌入层
        self.constant_embedding = nn.Embedding(len(vocab), embed_size)
        # 加载预训练的100维GloVe词向量，并分别初始化嵌入层embedding和constant_embedding，前者参与训练，而后者权重固定
        # 因此卷积层的输入通道个数为2 * embed_size
        self.dropout = nn.Dropout(0.5)
        self.decoder = nn.Linear(sum(num_channels), 2)
        # 时序最大池化层没有权重，所以可以共用一个实例
        self.pool = GlobalMaxPool1d()
        self.convs = nn.ModuleList()  # 创建多个一维卷积层
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.append(nn.Conv1d(in_channels=2 * embed_size, 
                                       out_channels=c, 
                                       kernel_size=k))
    
    def forward(self, inputs):
        # inputs: (batch_size, seq_len)
        # embeddings: (batch_size, seq_len, 2 * embed_size)
        embeddings = torch.cat((self.embedding(inputs), self.constant_embedding(inputs)), dim=2)
        # 根据Conv1D要求的输入格式，将词向量维，即一维卷积层的通道维(即词向量那一维)，变换到前一维
        # embeddings: (batch_size, 2 * embed_size, seq_len)
        embeddings = embeddings.permute(0, 2, 1)
        # conv(embeddings): (batch_size, out_channels, out_seq_len)
        # self.pool(F.relu(conv(embeddings))): (batch_size, out_channels, 1)
        # self.pool(F.relu(conv(embeddings))).squeeze(-1): (batch_size, out_channels)
        # 最后将他们在输出通道维上连接得到(batch_size, sum_out_channels)
        encoding = torch.cat([
            self.pool(F.relu(conv(embeddings))).squeeze(-1) for conv in self.convs
        ], dim=1)
        # 应用丢弃法后使用全连接层得到输出
        outputs = self.decoder(self.dropout(encoding))
        return outputs

In [32]:
embed_size, kernel_sizes, nums_channels = 100, [3, 4, 5], [100, 100, 100]
net = TextCNN(vocab, embed_size, kernel_sizes, nums_channels)

**训练并评价模型**：

In [36]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
my_utils.general_train(net, train_iter, test_iter, loss, num_epochs, batch_size, 
                       params=None, lr=None, optimizer=optimizer)

epoch 1, loss 0.0100, train acc 0.662, test acc 0.796
epoch 2, loss 0.0075, train acc 0.767, test acc 0.832
epoch 3, loss 0.0064, train acc 0.813, test acc 0.853
epoch 4, loss 0.0052, train acc 0.858, test acc 0.862
epoch 5, loss 0.0038, train acc 0.900, test acc 0.868


In [37]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great'])

'positive'